In [ ]:
import os
import re
import io
import sys
import glob
import enum
import json
import dask
import xlrd
import base64
import time
import shutil
import imageio
import requests
import datetime
import urllib
import psycopg2
import numpy as np
import pandas as pd
import skimage

import cProfile
import io
import pstats
import contextlib

import tifffile
import seaborn as sns
import matplotlib as mpl

import dask.diagnostics
import sqlalchemy as db
import sqlalchemy.orm
import sqlalchemy.ext.declarative
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 1

sys.path.append('../..')
%aimport opencell.imaging.managers
%aimport opencell.imaging.processors

from opencell.database import models
from opencell.database import operations
from opencell.database import utils as db_utils

In [ ]:
url = db_utils.url_from_credentials('../../db-credentials-test.json')
url = db_utils.url_from_credentials('../../db-credentials-dev.json')
# url = db_utils.url_from_credentials('../../db-credentials-cap.json')

engine = db.create_engine(url, echo=False)
session_factory = db.orm.sessionmaker(bind=engine)
Session = db.orm.scoped_session(session_factory)
url

In [ ]:
@contextlib.contextmanager
def profiled():
    pr = cProfile.Profile()
    pr.enable()
    yield
    pr.disable()
    s = io.StringIO()
    ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
    ps.print_stats()
    # uncomment this to see who's calling what
    # ps.print_callers()
    print(s.getvalue())

In [ ]:
# original pulldown query
pulldown_id = 562
query = (
    Session.query(models.MassSpecPulldown)
    .options(
        db.orm.joinedload(models.MassSpecPulldown.hits)
            .joinedload(models.MassSpecHit.protein_group)
            .joinedload(models.MassSpecProteinGroup.crispr_designs),
        db.orm.joinedload(models.MassSpecPulldown.hits)
            .joinedload(models.MassSpecHit.protein_group)
            .joinedload(models.MassSpecProteinGroup.uniprot_metadata),
    )
    .filter(models.MassSpecPulldown.id == pulldown_id)
)

start = time.time()
result = query.all()
end = time.time()
end - start, len(result)

In [ ]:
# get everything, but just for significant hits
pulldown_id = 562
query = (
    Session.query(models.MassSpecHit)
    .options(
        db.orm.joinedload(models.MassSpecHit.protein_group, innerjoin=True)
            .joinedload(models.MassSpecProteinGroup.crispr_designs),
        db.orm.joinedload(models.MassSpecHit.protein_group, innerjoin=True)
            .joinedload(models.MassSpecProteinGroup.uniprot_metadata),
    )
    .filter(models.MassSpecHit.pulldown_id == pulldown_id)
    .filter(db.or_(models.MassSpecHit.is_minor_hit == True, models.MassSpecHit.is_significant_hit == True))
)

start = time.time()
result = query.all()
end = time.time()
end - start, len(result)

In [ ]:
float('%0.2f' % .123456789)

In [ ]:
# get the pval and enrichment for all non-significant hits
pulldown_id = 562
query = (
    Session.query(models.MassSpecHit.pval, models.MassSpecHit.enrichment)
    .filter(models.MassSpecHit.pulldown_id == pulldown_id)
    .filter(models.MassSpecHit.is_minor_hit == False)
    .filter(models.MassSpecHit.is_significant_hit == False)
)

start = time.time()
result = query.all()
end = time.time()
end - start, len(result)

In [ ]:
with profiled():
    query.one()